In [1]:
!pip install opendatasets

In [2]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: saikatjana12
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis


100%|██████████| 1.99M/1.99M [00:00<00:00, 280MB/s]

In [3]:
import pandas as pd
df=pd.read_csv("/content/twitter-entity-sentiment-analysis/twitter_training.csv")
df.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [4]:
df.columns.tolist()

['2401',
 'Borderlands',
 'Positive',
 'im getting on borderlands and i will murder you all ,']

In [ ]:
df['Positive'].unique()

array(['Positive', 'Neutral', 'Negative', 'Irrelevant'], dtype=object)

In [14]:
x=df[df['Positive']=='Positive']

In [16]:
x['im getting on borderlands and i will murder you all ,'].iloc[1]

'im getting on borderlands and i will kill you all,'

In [ ]:
df=df.drop(['2401','Borderlands'],axis=1)

In [ ]:
df.head()

,Positive,"im getting on borderlands and i will murder you all ,"
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...


In [ ]:
df=df.rename(columns={
    'Positive':'sentiment',
    'im getting on borderlands and i will murder you all ,':'comment'
})

In [ ]:
df.head()

,sentiment,comment
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...


In [ ]:
df.shape

(74681, 2)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
level_encoder=LabelEncoder()
df['sentiment']=level_encoder.fit_transform(df['sentiment'])

In [ ]:
df['sentiment'].unique()

array([3, 2, 1, 0])

In [ ]:
df

,sentiment,comment
0,3,I am coming to the borders and I will kill you...
1,3,im getting on borderlands and i will kill you ...
2,3,im coming on borderlands and i will murder you...
3,3,im getting on borderlands 2 and i will murder ...
4,3,im getting into borderlands and i can murder y...
...,...,...
74676,3,Just realized that the Windows partition of my...
74677,3,Just realized that my Mac window partition is ...
74678,3,Just realized the windows partition of my Mac ...
74679,3,Just realized between the windows partition of...


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["comment"], df["sentiment"], test_size=0.2, random_state=42
)

In [ ]:
train_texts.shape

(59744,)

In [ ]:
val_texts.shape

(14937,)

In [ ]:
train_labels.shape

(59744,)

In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
print(df['comment'].apply(type).value_counts())

comment
<class 'str'>    74681
Name: count, dtype: int64


In [ ]:
df['comment'] = df['comment'].astype(str)

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')
comments_list = df['comment'].tolist()
comment_embeddings = model.encode(comments_list)

In [ ]:
comment_embeddings.shape

(74681, 384)

In [ ]:
df['embedding'] = list(comment_embeddings)

In [ ]:
df=df.drop(['comment'],axis=1)

In [ ]:
df

,sentiment,embedding
0,3,"[0.058729053, 0.030680636, 0.0013802312, -0.05..."
1,3,"[0.050012168, -0.016531097, -0.069782175, -0.0..."
2,3,"[0.04015841, -0.0024829335, -0.08179742, -0.06..."
3,3,"[0.04747411, 0.008014571, -0.08486219, -0.0580..."
4,3,"[0.059076127, 0.004042037, -0.116136104, -0.04..."
...,...,...
74676,3,"[0.013328244, -0.042734724, 0.09144183, 0.0663..."
74677,3,"[0.023357376, -0.054183993, 0.10240901, 0.0455..."
74678,3,"[0.014926678, -0.013800451, 0.1052194, 0.05665..."
74679,3,"[0.0032092133, -0.070200816, 0.107152455, 0.06..."


In [ ]:
x_train, x_test, y_train, y_test=train_test_split(df['embedding'],df['sentiment'],random_state=42,test_size=0.2)

In [ ]:
x_train[0].shape

(384,)

In [ ]:
import torch
import numpy as np

In [ ]:
X_train_stacked = np.array(x_train.tolist())
X_test_stacked = np.array(x_test.tolist())

In [ ]:
X_train_tensor = torch.tensor(X_train_stacked, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_stacked, dtype=torch.float32)

In [ ]:
X_train_tensor.size()

torch.Size([59744, 384])

In [ ]:
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

In [ ]:
torch.unique(y_train_tensor)

tensor([0, 1, 2, 3])

In [ ]:
print(y_train_tensor.dtype)

torch.int64


In [ ]:
y_train_tensor.size()

torch.Size([59744])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
def scaled_dot_product(q, k, v, mask=None):
    """Calculates the scaled dot-product attention."""
    d_k = q.size()[-1]
    scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k)
    if mask is not None:
        scaled += mask
    attention = F.softmax(scaled, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention

class MultiHeadAttention(nn.Module):
    """Multi-Head Attention mechanism."""
    def __init__(self, d_model, num_heads):
        super().__init__()
        # Check to prevent architectural errors before model creation
        assert d_model % num_heads == 0, f"d_model ({d_model}) must be divisible by num_heads ({num_heads})"

        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.qkv_layer = nn.Linear(d_model, 3 * d_model)
        self.linear_layer = nn.Linear(d_model, d_model)

    def forward(self, x, mask=None):
        batch_size, max_sequence_length, d_model = x.size()
        qkv = self.qkv_layer(x)
        qkv = qkv.reshape(batch_size, max_sequence_length, self.num_heads, 3 * self.head_dim)
        qkv = qkv.permute(0, 2, 1, 3)
        q, k, v = qkv.chunk(3, dim=-1)
        values, attention = scaled_dot_product(q, k, v, mask)

        # --- THIS IS THE FIX ---
        # The 'values' tensor has shape (batch, heads, sequence, head_features).
        # We must permute it back to (batch, sequence, heads, head_features)
        # before we can reshape it to combine the last two dimensions.
        values = values.permute(0, 2, 1, 3).contiguous()

        # Now we can safely reshape
        values = values.reshape(batch_size, max_sequence_length, self.num_heads * self.head_dim)

        out = self.linear_layer(values)
        return out

In [ ]:
class LayerNormalization(nn.Module):
    """Layer Normalization module."""
    def __init__(self, parameters_shape, eps=1e-5):
        super().__init__()
        self.parameters_shape = parameters_shape
        self.eps = eps
        self.gamma = nn.Parameter(torch.ones(parameters_shape))
        self.beta = nn.Parameter(torch.zeros(parameters_shape))

    def forward(self, inputs):
        dims = [-(i + 1) for i in range(len(self.parameters_shape))]
        mean = inputs.mean(dim=dims, keepdim=True)
        var = ((inputs - mean) ** 2).mean(dim=dims, keepdim=True)
        std = (var + self.eps).sqrt()
        y = (inputs - mean) / std
        out = self.gamma * y + self.beta
        return out

In [ ]:
class PositionwiseFeedForward(nn.Module):
    """Position-wise Feed-Forward network."""
    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden)
        self.linear2 = nn.Linear(hidden, d_model)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

In [ ]:
class EncoderLayer(nn.Module):
    """A single layer of the Transformer Encoder."""
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)
        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)

    def forward(self, x, mask=None):
        residual_x = x
        x = self.attention(x, mask=mask)
        x = self.dropout1(x)
        x = self.norm1(x + residual_x)
        residual_x = x
        x = self.ffn(x)
        x = self.dropout2(x)
        x = self.norm2(x + residual_x)
        return x

In [ ]:
class Encoder(nn.Module):
    """The full Transformer Encoder."""
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob, num_layers):
        super().__init__()
        self.layers = nn.Sequential(*[EncoderLayer(d_model, ffn_hidden, num_heads, drop_prob)
                                      for _ in range(num_layers)])
    def forward(self, x, mask=None):
        x = self.layers(x)
        return x


In [ ]:
class SentimentClassifier(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob, num_layers, num_classes):
        super().__init__()
        self.encoder = Encoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers)
        self.classifier_head = nn.Linear(d_model, num_classes)

    def forward(self, x):
        # Reshape input for the encoder: (batch_size, d_model) -> (batch_size, 1, d_model)
        x = x.unsqueeze(1)
        encoder_output = self.encoder(x)
        # Squeeze output for the classifier head: (batch_size, 1, d_model) -> (batch_size, d_model)
        encoder_output = encoder_output.squeeze(1)
        logits = self.classifier_head(encoder_output)
        return logits

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [ ]:
D_MODEL = 384  # This MUST match your data's feature dimension
NUM_HEADS = 8
FFN_HIDDEN = D_MODEL * 4  # A common practice
DROP_PROB = 0.1
NUM_LAYERS = 6 # Number of EncoderLayers

# Training parameters
BATCH_SIZE = 64
EPOCHS = 10
LR = 1e-4
NUM_CLASSES=4

In [ ]:
torch.unique(y_train_tensor)

tensor([0, 1, 2, 3])

In [ ]:
dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
train_loader

Error


In [ ]:
model = SentimentClassifier(
        d_model=D_MODEL,
        ffn_hidden=FFN_HIDDEN,
        num_heads=NUM_HEADS,
        drop_prob=DROP_PROB,
        num_layers=NUM_LAYERS,
        num_classes=NUM_CLASSES
    ).to(device)

AcceleratorError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
print("\n🚀 Starting Training...")

# --- Step 4: The Training Loop ---
for epoch in range(EPOCHS):
    model.train()  # Set the model to training mode

    # Initialize tracking variables
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    # --- Loop over batches ---
    for batch_inputs, batch_labels in train_loader:
        # Move data to the selected device (GPU/CPU)
        batch_inputs = batch_inputs.to(device)
        batch_labels = batch_labels.to(device)

        # 1. Zero the gradients
        optimizer.zero_grad()

        # 2. Forward pass
        outputs = model(batch_inputs)

        # 3. Compute loss
        loss = criterion(outputs, batch_labels)

        # 4. Backward pass
        loss.backward()

        # 5. Update weights
        optimizer.step()

        # --- Track statistics ---
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_samples += batch_labels.size(0)
        correct_predictions += (predicted == batch_labels).sum().item()

    # --- End of Epoch ---
    avg_loss = total_loss / len(train_loader)
    accuracy = (correct_predictions / total_samples) * 100

    print(f"Epoch [{epoch+1}/{EPOCHS}] "
          f"Loss: {avg_loss:.4f} | Accuracy: {accuracy:.2f}%")

print("\n✅ Training Finished!")



🚀 Starting Training...


AcceleratorError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [17]:
D_MODEL = 384  # This MUST match your data's feature dimension
NUM_HEADS = 8
FFN_HIDDEN = D_MODEL * 4  # A common practice
DROP_PROB = 0.1
NUM_LAYERS = 6 # Number of EncoderLayers

# Training parameters
BATCH_SIZE = 64
EPOCHS = 10
LR = 1e-4
NUM_CLASSES=4

In [18]:
model = SentimentClassifier(
        d_model=D_MODEL,
        ffn_hidden=FFN_HIDDEN,
        num_heads=NUM_HEADS,
        drop_prob=DROP_PROB,
        num_layers=NUM_LAYERS,
        num_classes=NUM_CLASSES
    )

NameError: name 'SentimentClassifier' is not defined